In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('loanData.csv')

#sorting the data based on the FICO score
df = df.sort_values(by='fico_score')

#choosing the number of buckets here (I want 5 for no specific reason)
num_buckets = 5



In [10]:
#creating buckets
bins = np.linspace(df['fico_score'].min(), df['fico_score'].max(), num_buckets + 1)
df['rating_bucket'] = np.digitize(df['fico_score'], bins) - 1
df['rating_bucket'] = pd.qcut(df['fico_score'], num_buckets, labels=False)

In [11]:
#assigning rating to the buckets
df['rating'] = num_buckets - df['rating_bucket']
print(df[['fico_score', 'rating_bucket', 'rating']])

      fico_score  rating_bucket  rating
2092         408              0       5
6556         409              0       5
7001         418              0       5
5521         425              0       5
2629         438              0       5
...          ...            ...     ...
703          828              4       1
7575         831              4       1
4768         831              4       1
9660         835              4       1
2659         850              4       1

[10000 rows x 3 columns]


In [12]:
#ensuring everything is grouped correctly
print(df[['fico_score', 'rating_bucket', 'rating']].groupby('rating_bucket').describe())

              fico_score                                                     \
                   count        mean        std    min    25%    50%    75%   
rating_bucket                                                                 
0                 2050.0  552.611707  30.204872  408.0  537.0  561.0  576.0   
1                 1971.0  606.751395  10.228908  588.0  598.0  607.0  616.0   
2                 1989.0  638.579688   8.670721  624.0  631.0  639.0  646.0   
3                 1997.0  670.346520  10.112201  654.0  662.0  670.0  679.0   
4                 1993.0  721.524837  27.417593  689.0  700.0  714.0  735.0   

                      rating                                     
                 max   count mean  std  min  25%  50%  75%  max  
rating_bucket                                                    
0              587.0  2050.0  5.0  0.0  5.0  5.0  5.0  5.0  5.0  
1              623.0  1971.0  4.0  0.0  4.0  4.0  4.0  4.0  4.0  
2              653.0  1989.0  3.0  0.

In [13]:
#Calculating MSE and Log Likelihood
bucket_means = df.groupby('rating_bucket')['fico_score'].mean()
mse = ((df['fico_score'] - df['rating_bucket'].map(bucket_means))**2).mean()
print(f'Mean Squared Error: {mse}')

bucket_stats = df.groupby('rating_bucket').agg(total=('default', 'count'), defaults=('default', 'sum'))
bucket_stats['p'] = bucket_stats['defaults'] / bucket_stats['total']
log_likelihood = (bucket_stats['total'] * np.log(bucket_stats['p'].replace(0, np.nan))).sum()
print(f'Log-Likelihood: {log_likelihood}')

Mean Squared Error: 392.6494950978713
Log-Likelihood: -19071.003016272032
